In [1]:
import cv2
import pyrealsense2 as rs
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
%gui tk

In [2]:
def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    return file_path

# Setup (Realsense)

In [3]:
def read_bag_file(file_name, real_time=False):
    """
    Read a bag file recorded from a RealSense camera.

    Parameters:
        file_name (str): the path to the bag file.
        real_time (bool): if True, the playback is in real time (frames may be dropped).
    """
    pipe = rs.pipeline()
    cfg = rs.config()
    cfg.enable_device_from_file(file_name, repeat_playback=False)
    profile = pipe.start(cfg)
    playback = profile.get_device().as_playback()
    playback.set_real_time(real_time) # False: no frame drop
    
    # Get the frame shape
    frames = pipe.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    frame_shape = (depth_frame.get_height(), depth_frame.get_width())

    return pipe, cfg, profile, playback, frame_shape

In [4]:
def post_process_depth_frame(depth_frame, min_distance=0, max_distance=4, decimation_magnitude = 1.0, spatial_magnitude = 2.0, spatial_smooth_alpha = 0.5, spatial_smooth_delta = 20, temporal_smooth_alpha = 0.4, temporal_smooth_delta = 20, fill_hole = False):
    """
    Apply post processing filters to the depth frame of a RealSense camera.
    More information about the filters can be found at:
    - https://dev.intelrealsense.com/docs/post-processing-filters
    - https://github.com/IntelRealSense/librealsense/blob/jupyter/notebooks/depth_filters.ipynb
    - https://intelrealsense.github.io/librealsense/doxygen/namespacers2.html
    """
    # Post processing possible only on the depth_frame
    assert (depth_frame.is_depth_frame())

    # Available filters
    decimation_filter = rs.decimation_filter()
    threshold_filter = rs.threshold_filter()
    depth_to_disparity = rs.disparity_transform(True)
    spatial_filter = rs.spatial_filter()
    temporal_filter = rs.temporal_filter()
    disparity_to_depth = rs.disparity_transform(False)
    hole_filling = rs.hole_filling_filter(1) # https://intelrealsense.github.io/librealsense/doxygen/classrs2_1_1hole__filling__filter.html

    # Apply the control parameters for the filters
    decimation_filter.set_option(rs.option.filter_magnitude, decimation_magnitude)
    threshold_filter.set_option(rs.option.min_distance, min_distance)
    threshold_filter.set_option(rs.option.max_distance, max_distance)
    spatial_filter.set_option(rs.option.filter_magnitude, spatial_magnitude)
    spatial_filter.set_option(rs.option.filter_smooth_alpha, spatial_smooth_alpha)
    spatial_filter.set_option(rs.option.filter_smooth_delta, spatial_smooth_delta)
    temporal_filter.set_option(rs.option.filter_smooth_alpha, temporal_smooth_alpha)
    temporal_filter.set_option(rs.option.filter_smooth_delta, temporal_smooth_delta)

    # Apply the filters
    # Post processing order : https://dev.intelrealsense.com/docs/post-processing-filters
    # Depth Frame >> Decimation Filter >> Depth2Disparity Transform >> Spatial Filter >> Temporal Filter >> Disparity2Depth Transform >> Hole Filling Filter >> Filtered Depth
    filtered_frame = decimation_filter.process(depth_frame)
    filtered_frame = threshold_filter.process(filtered_frame)
    filtered_frame = depth_to_disparity.process(filtered_frame)
    filtered_frame = spatial_filter.process(filtered_frame)
    filtered_frame = temporal_filter.process(filtered_frame)
    filtered_frame = disparity_to_depth.process(filtered_frame)
    if fill_hole:
        filtered_frame = hole_filling.process(filtered_frame)
    
    # Cast to depth_frame so that we can use the get_distance method afterwards
    depth_frame_filtered = filtered_frame.as_depth_frame()

    return depth_frame_filtered

In [9]:
colorizer = rs.colorizer()
align = rs.align(rs.stream.color)

def process_frames(frames, post_process = True):
    """
    Process the depth and color frames from a RealSense camera.
    Align the depth frame to the color frame, colorize the depth frame and convert the frames to images.
    """
    # Align the depth frame to the color frame
    aligned_frames = align.process(frames)

    # Get aligned frames
    depth_frame = aligned_frames.get_depth_frame()
    color_frame = aligned_frames.get_color_frame()

    # Validate that both frames are valid
    if not depth_frame or not color_frame:
        return None, None, None, None

    # Post process is not included in the BAG file, so we need to apply it
    if post_process:
        depth_frame = post_process_depth_frame(depth_frame, fill_hole=True)

    # Colorize the depth frame
    depth_color_frame = colorizer.colorize(depth_frame)

    # Convert frames to images
    depth_color_image = np.asanyarray(depth_color_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    return color_frame, depth_frame, color_image, depth_color_image

# Visualize the data

In [6]:
# Read the bag file
bag_file = select_file()
if not bag_file:
    bag_file = "C:\\Users\\Robin\\Documents\\Stage2024\\Dataset\\bag_records\\2_people_height_estimation.bag"
pipe, cfg, profile, playback, frame_shape = read_bag_file(bag_file)
duration = playback.get_duration()
print("Video duration: ", duration)
print("Frame shape: ", frame_shape)

Video duration:  0:00:43.503743
Frame shape:  (480, 848)


In [7]:
# Show the full stream
pipe, cfg, profile, playback, frame_shape = read_bag_file(bag_file)
num_frames = 0
wait_key = 1

try:
    while True:
        # Get frameset of color and depth
        frames = pipe.wait_for_frames()

        # Process the frames
        color_frame, depth_frame, color_image, depth_color_image = process_frames(frames, True)

        # Verify that the frames are valid
        if color_frame is None or depth_frame is None:
            continue
        
        cv2.imshow("Color Image", color_image)
        cv2.imshow("Depth Image", depth_color_image)

        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press s to save the color image
        if key == ord('s'):
            cv2.imwrite('color_image.png', color_image)

        # Press d to view the video frame by frame
        if key == ord('d'):
            wait_key = 1 if wait_key == 0 else 0

        num_frames += 1

# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")

# Release the stream 
finally:
    cv2.destroyAllWindows()
    pipe.stop()

print("Total number of frames: ", num_frames)

Total number of frames:  73


# Extract height of the participants

In [8]:
# Setup the aruco detector
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_ARUCO_MIP_36h12)
aruco_params = cv2.aruco.DetectorParameters()

aruco_params.errorCorrectionRate = 0.2
aruco_params.polygonalApproxAccuracyRate = 0.05
aruco_params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_CONTOUR
# aruco_params.minMarkerPerimeterRate = 0.01

detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params) 

In [12]:
camera_height = 2.568 # meters

pipe, cfg, profile, playback, frame_shape = read_bag_file(bag_file)
num_frames = 0
wait_key = 1

# Data structure to store the a list of height for every aruco marker id
heights = {}

try:
    while True:
        # Get frameset of color and depth
        frames = pipe.wait_for_frames()

        # Process the frames
        color_frame, depth_frame, color_image, depth_color_image = process_frames(frames)
        #color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)

        # Detect the aruco markers
        corners, ids, rejected = detector.detectMarkers(color_image)
        cv2.aruco.drawDetectedMarkers(color_image, corners, ids)

        for k in range(len(corners)):
            id = ids[k][0]
            c = corners[k][0]

            # Calculate the distance using the center of the aruco marker
            x = int(c[:, 0].sum() / 4)
            y = int(c[:, 1].sum() / 4)
            distance = depth_frame.get_distance(x, y)

            # Calculate the height
            height = camera_height - distance

            # Display the height with an offset to avoid overlap
            text_position_y = 30 + k * 40
            cv2.putText(color_image, "ID: {} Height: {:.2f}m".format(id, height), (10, text_position_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            # Store the height for the aruco marker id
            if not id in heights:
                heights[id] = []
                
            heights[id].append(height)

        cv2.imshow("Output", depth_color_image)
        cv2.imshow("Color Image", color_image)
        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press d to view the video frame by frame
        if key == ord('d'):
            wait_key = 1 if wait_key == 0 else 0

        num_frames += 1
           
# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")

# Release the stream    
finally:
    cv2.destroyAllWindows()
    pipe.stop()

In [10]:
def remove_outliers(data, m=2):
    """
    Remove outliers using Z-score method.
    """
    data = np.array(data)
    mean = np.mean(data)
    std = np.std(data)
    filtered_data = [x for x in data if abs(x - mean) / std < m]
    return filtered_data

def calculate_mean_height(heights):
    mean_heights = {}

    for key, height_list in heights.items():
        filtered_heights = remove_outliers(height_list)
        if filtered_heights:
            mean_heights[key] = np.mean(filtered_heights)

    return mean_heights

# Remove outliers and calculate the mean height for each aruco marker id
mean_heights = calculate_mean_height(heights)
print(mean_heights)

{}


# Projection of the point on the floor

In [11]:
# Matching points on the floor, 130cm and 170cm
points_floor = np.array([
    [1042, 737, 0],
    [1161, 747, 0],
    [1287, 760, 0],
    [1405, 771, 0],
    [1025, 861, 0],
    [1148, 870, 0],
    [1274, 882, 0],
    [1397, 891, 0],
    [1011, 986, 0],
    [1138, 997, 0],
    [1265, 1007, 0],
    [1392, 1018, 0],
    [996, 1111, 0],
    [1128, 1120, 0],
    [1257, 1128, 0],
    [1382, 1134, 0]
])

points_130 = np.array([
    [805, 395, 130],
    [1060, 420, 130],
    [1320, 449, 130],
    [1554, 473, 130],
    [777, 651, 130],
    [1040, 677, 130],
    [1301, 702, 130],
    [1549, 731, 130],
    [746, 914, 130],
    [1024, 937, 130],
    [1286, 960, 130],
    [1532, 976, 130],
    [722, 1130, 130],
    [1017, 1178, 130],
    [1257, 1179, 130],
    [1507, 1208, 130]
])

points_170 = np.array([
    [598, 91, 170],
    [970, 118, 170],
    [1346, 167, 170],
    [1693, 206, 170],
    [537, 445, 170],
    [931, 489, 170],
    [1323, 536, 170],
    [1685, 576, 170],
    [481, 842, 170],
    [906, 873, 170],
    [1301, 916, 170],
    [1664, 933, 170],
    [444, 1172, 170],
    [895, 1240, 170],
    [1261, 1241, 170],
    [1634, 1278, 170]
])

In [12]:
points_floor_file = "C:\\Users\\Robin\\Documents\Stage2024\\Dataset\\points_measurements_projection\\mini_experiment\\Image__2024-03-20__16-55-35.png"
image = cv2.imread(points_floor_file)

for pf, p130, p170 in zip(points_floor, points_130, points_170):
    cv2.line(image, pf[:2], p130[:2], (255, 0, 0), 1)
    cv2.line(image, p130[:2], p170[:2], (0, 255, 0), 1)

cv2.namedWindow('Projection Lines', cv2.WINDOW_NORMAL)
cv2.imshow('Projection Lines', image)
cv2.resizeWindow('Projection Lines', 864, 648)
cv2.waitKey()
cv2.destroyAllWindows()

In [13]:
# Compute the 3 homographies
points_floor_2d = points_floor[:, :2]
points_130_2d = points_130[:, :2]
points_170_2d = points_170[:, :2]

H_0, _ = cv2.findHomography(points_floor_2d, points_floor_2d) # Identity matrix (hopefully :))
H_130, _ = cv2.findHomography(points_130_2d, points_floor_2d)
H_170, _ = cv2.findHomography(points_170_2d, points_floor_2d)

In [14]:
def interpolate_homography1(z, H_0, H_130, H_170):
    """
    Interpolate the homography matrix for a given height, between 0 and 210 cm.
    """
    if z <= 130: # linear interpolation between 0 and 130
        alpha = z / 130 # /40 to normalize the alpha value (0 <= z <= 130)
        return (1 - alpha) * H_0 + alpha * H_130
    
    elif z <= 170: # linear interpolation between 130 and 170
        alpha = (z - 130) / 40 # /40 to normalize the alpha value (130 < z <= 170)
        return (1 - alpha) * H_130 + alpha * H_170
    
    else: # extrapolation between 170 and 210
        # Basically, from 170 cm, the transformation continues to change in the same way as it did between 130 cm and 170 cm
        H_210 =  H_170 + (H_170 - H_130)
        alpha = (z - 170) / 40 # /40 to normalize the alpha value (170 < z <= 210)
        return (1 - alpha) * H_170 + alpha * H_210

def project_point(point, z, H):
    """
    Project a 3D point on the floor, given its 3D coordinates and the homography matrix.

    Parameters:
        point (np.array): the 2D point coordinates [x, y].
        z (float): the height of the point.
    """
    point_homogeneous = np.hstack([point, 1])
    projected_point_homogeneous = H @ point_homogeneous
    projected_point = projected_point_homogeneous[:2] / projected_point_homogeneous[2]
    return projected_point

In [15]:
# Small example to test the homography interpolation
image = cv2.imread(points_floor_file)

cv2.namedWindow('Projection Points', cv2.WINDOW_NORMAL)
cv2.imshow('Projection Points', image)
cv2.resizeWindow('Projection Points', 864, 648)

# Draw the points at 170 cm
for point in points_170.astype(int):
    cv2.circle(image, tuple(point[:2]), 10, (255, 0, 0), -1)

def on_trackbar(z):
    z = int(z)
    H = interpolate_homography1(z, H_0, H_130, H_170)

    projected_points = []

    for point in points_170:
        point = point[:2]
        projected_point = project_point(point, z, H)
        projected_points.append(projected_point)

    image_copy = image.copy()
    for point in projected_points:
        point = point.astype(int)
        cv2.circle(image_copy, tuple(point), 10, (0, 0, 255), -1)
        
    cv2.imshow('Projection Points', image_copy)

cv2.createTrackbar('Height', 'Projection Points', 170, 210, on_trackbar)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
baslar_file = select_file()
if not baslar_file:
    baslar_file = "C:\\Users\\Robin\\Documents\\Stage2024\\Dataset\\evacuation_experiments\\solo_experiment\\rec.mp4"
cap = cv2.VideoCapture(baslar_file)
wait_key = 1
height = 180 # cm
projected_points = []
aruco_ids = set()

while cap.isOpened():
    ret, image = cap.read()

    if ret:
        # Detect the aruco markers
        corners, ids, rejected = detector.detectMarkers(image)
        cv2.aruco.drawDetectedMarkers(image, corners, ids)

        for k in range(len(corners)):
            id = ids[k][0]
            c = corners[k][0]

            aruco_ids.add(id)

            # Center of the aruco marker
            x = int(c[:, 0].sum() / 4)
            y = int(c[:, 1].sum() / 4)

            # Project the point on the floor
            point = (x, y)
            H = interpolate_homography1(height, H_0, H_130, H_170)
            projected_point = project_point(point, height, H)
            projected_points.append(projected_point)

            cv2.circle(image, point, 10, (0, 0, 255), -1)
        
        # Draw all the projected points to get the trajectory
        for projected_point in projected_points:
            point = projected_point.astype(int)
            cv2.circle(image, tuple(point), 10, (0, 255, 0), -1)

        cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
        cv2.imshow('Frame', image)
        cv2.resizeWindow('Frame', 864, 648)
        
        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press d to view the video frame by frame
        if key == ord('d'):
            wait_key = 1 if wait_key == 0 else 0
    else:
        break

cap.release()
cv2.destroyAllWindows()

# Estimate exit times

## 1. Estimate heights

In [17]:
""" camera_height = 2.568 # meters, ground truth

pipe, cfg, profile, playback, frame_shape = read_bag_file(bag_file)
num_frames = 0

# Data structure to store the a list of height for every aruco marker id
heights = {}

try:
    while True:
        # Get frameset of color and depth
        frames = pipe.wait_for_frames()

        # Process the frames
        color_frame, depth_frame, color_image, depth_color_image = process_frames(frames)

        # Detect the aruco markers
        corners, ids, rejected = detector.detectMarkers(color_image)

        for k in range(len(corners)):
            id = ids[k][0]
            c = corners[k][0]

            # Calculate the distance using the center of the aruco marker
            x = int(c[:, 0].sum() / 4)
            y = int(c[:, 1].sum() / 4)
            distance = depth_frame.get_distance(x, y)

            # Calculate the height
            height = camera_height - distance

            # Store the height for the aruco marker id
            if not id in heights:
                heights[id] = []
                
            heights[id].append(height)

        num_frames += 1
           
# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")
        
finally:
    # Stop streaming
    pipe.stop()

print("Total number of frames: ", num_frames)

# Remove outliers and calculate the mean height for each aruco marker id
mean_heights = calculate_mean_height(heights)
print(mean_heights) """

' camera_height = 2.568 # meters, ground truth\n\npipe, cfg, profile, playback, frame_shape = read_bag_file(bag_file)\nnum_frames = 0\n\n# Data structure to store the a list of height for every aruco marker id\nheights = {}\n\ntry:\n    while True:\n        # Get frameset of color and depth\n        frames = pipe.wait_for_frames()\n\n        # Process the frames\n        color_frame, depth_frame, color_image, depth_color_image = process_frames(frames)\n\n        # Detect the aruco markers\n        corners, ids, rejected = detector.detectMarkers(color_image)\n\n        for k in range(len(corners)):\n            id = ids[k][0]\n            c = corners[k][0]\n\n            # Calculate the distance using the center of the aruco marker\n            x = int(c[:, 0].sum() / 4)\n            y = int(c[:, 1].sum() / 4)\n            distance = depth_frame.get_distance(x, y)\n\n            # Calculate the height\n            height = camera_height - distance\n\n            # Store the height for t

In [18]:
# Hardcoded heights because the realsense camera was not properly placed
aruco_ids = {33, 17, 87, 24, 25}
mean_heights = {key: 1.75 for key in aruco_ids}
print(mean_heights)

{33: 1.75, 17: 1.75, 87: 1.75, 24: 1.75, 25: 1.75}


## 2. 

In [19]:
baslar_file = select_file()
if not baslar_file:
    baslar_file = "C:\\Users\\Robin\\Documents\\Stage2024\\Dataset\\evacuation_experiments\\group_experiment\\first.mp4"
cap = cv2.VideoCapture(baslar_file)
wait_key = 1
num_frames = 0

# Dictionary to store the projected points for each aruco marker id, and the color for each id, only for visualization
projected_points = {}
color_points = {}
for id, _ in mean_heights.items():
    color_points[id] = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))

# Data structure to store the enter and exit frame for each aruco marker id
enter_frame = {}
exit_frame = {}
for id, _ in mean_heights.items():
    enter_frame[id] = -1
    exit_frame[id] = -1

# Y coordinate of the "exit line"
y_exit = 1130

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        # Detect the aruco markers
        corners, ids, rejected = detector.detectMarkers(frame)
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        for k in range(len(corners)):
            id = ids[k][0]
            c = corners[k][0]

            # Center of the aruco marker
            x = int(c[:, 0].sum() / 4)
            y = int(c[:, 1].sum() / 4)

            # Project the point on the floor
            point = (x, y)
            height = mean_heights[id] * 100 # cm
            H = interpolate_homography1(height, H_0, H_130, H_170)
            projected_point = project_point(point, height, H)

            # Store the projected point for the aruco marker id
            if not id in projected_points:
                projected_points[id] = []
                
            projected_points[id].append(projected_point)

            # Update the enter and exit frame
            if enter_frame[id] == -1:
                enter_frame[id] = num_frames

            if y >= y_exit:
                exit_frame[id] = num_frames

            # Draw a circle at the center of the aruco marker
            cv2.circle(frame, point, 10, (0, 0, 255), -1)
        
        # Draw all the projected points to get the trajectory
        for id, points in projected_points.items():
            for projected_point in points:
                point = projected_point.astype(int)
                if point[1] < y_exit:
                    cv2.circle(frame, tuple(point), 10, color_points[id], -1)

        cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
        cv2.imshow('Frame', frame)
        cv2.resizeWindow('Frame', 864, 648)
        
        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press d to view the video frame by frame
        if key == ord('d'):
            if wait_key == 0:
                wait_key = 1
            else:
                wait_key = 0

        num_frames += 1

    else:
        break

cap.release()
cv2.destroyAllWindows()


In [20]:
print("Enter frames: ", enter_frame)
print("Exit frames: ", exit_frame)

# Compute the difference between the enter and exit frame, to estimate the exit time for each id
exit_time = {}
basler_fps = 24

for id, enter in enter_frame.items():
    exit = exit_frame[id]
    if enter != -1 and exit != -1:
        exit_time[id] = (exit - enter) / basler_fps

# Print exit times in seconds
for id, time in exit_time.items():
    print("ID: {} Exit time: {:.2f}s".format(id, time))

Enter frames:  {33: -1, 17: -1, 87: -1, 24: -1, 25: -1}
Exit frames:  {33: -1, 17: -1, 87: -1, 24: -1, 25: -1}


# Artificial wall

## 1. Synchronize the 2 streams

In [21]:
bag_file_master = select_file()
if not bag_file_master:
    bag_file_master = "C:\\Users\\Robin\\Documents\\Stage2024\\Dataset\\bag_records\\realsense_sync\\master2.bag"
pipe_master, cfg_master, profile_master, playback_master, frame_shape_master = read_bag_file(bag_file_master)

bag_file_slave = select_file()
if not bag_file_slave:
    bag_file_slave = "C:\\Users\\Robin\\Documents\\Stage2024\\Dataset\\bag_records\\realsense_sync\\slave2.bag"
pipe_slave, cfg_slave, profile_slave, playback_slave, frame_shape_slave = read_bag_file(bag_file_slave)

In [22]:
frames_master = pipe_master.wait_for_frames()
frames_slave = pipe_slave.wait_for_frames()

depth_frame_master = frames_master.get_depth_frame()
depth_frame_slave = frames_slave.get_depth_frame()

depth_timestamp_master = depth_frame_master.get_timestamp()
depth_timestamp_slave = depth_frame_slave.get_timestamp()
timestamp_diff = abs(depth_timestamp_master - depth_timestamp_slave)

print("Master timestamp: ", depth_frame_master.get_timestamp())
print("Slave timestamp: ", depth_frame_slave.get_timestamp())
print("Timestamp difference: {} ms".format(timestamp_diff))

Master timestamp:  1720093892611.5762
Slave timestamp:  1720093896731.1372
Timestamp difference: 4119.56103515625 ms


In [23]:
# Let the master camera catch up with the slave camera
while timestamp_diff > 30: # 30 ms difference
    frames_master = pipe_master.wait_for_frames()
    depth_frame_master = frames_master.get_depth_frame()
    depth_timestamp_master = depth_frame_master.get_timestamp()

    timestamp_diff = abs(depth_timestamp_master - depth_timestamp_slave)

depth_timestamp_master = depth_frame_master.get_timestamp()
depth_timestamp_slave = depth_frame_slave.get_timestamp()
timestamp_diff = abs(depth_timestamp_master - depth_timestamp_slave)

print("Master timestamp: ", depth_frame_master.get_timestamp())
print("Slave timestamp: ", depth_frame_slave.get_timestamp())
print("Timestamp difference: {} ms".format(timestamp_diff))

Master timestamp:  1720093896712.5283
Slave timestamp:  1720093896731.1372
Timestamp difference: 18.60888671875 ms


In [24]:
def read_and_synchronize_bag_files(file_name_master, file_name_slave):
    """
    Read and synchronize two bag files recorded from two RealSense cameras.
    Synchronize the master and slave streams based on the global timestamps.
    """
    # Master stream
    pipe_master = rs.pipeline()
    cfg_master = rs.config()
    cfg_master.enable_device_from_file(file_name_master, repeat_playback=False)
    profile_master = pipe_master.start(cfg_master)
    playback_master = profile_master.get_device().as_playback()
    playback_master.set_real_time(False) # False: no frame drop
    
    # Slave stream
    pipe_slave = rs.pipeline()
    cfg_slave = rs.config()
    cfg_slave.enable_device_from_file(file_name_slave, repeat_playback=False)
    profile_slave = pipe_slave.start(cfg_slave)
    playback_slave = profile_slave.get_device().as_playback()
    playback_slave.set_real_time(False) # False: no frame drop

    # Sync the master and slave streams
    frames_master = pipe_master.wait_for_frames()
    frames_slave = pipe_slave.wait_for_frames()

    depth_frame_master = frames_master.get_depth_frame()
    depth_frame_slave = frames_slave.get_depth_frame()

    depth_timestamp_master = depth_frame_master.get_timestamp()
    depth_timestamp_slave = depth_frame_slave.get_timestamp()
    timestamp_diff = abs(depth_timestamp_master - depth_timestamp_slave)

    while timestamp_diff > 30: # 30 ms difference
        frames_master = pipe_master.wait_for_frames()
        depth_frame_master = frames_master.get_depth_frame()
        depth_timestamp_master = depth_frame_master.get_timestamp()

        timestamp_diff = abs(depth_timestamp_master - depth_timestamp_slave)

    return pipe_master, pipe_slave, timestamp_diff

In [25]:
pipe_master, pipe_slave, timestamp_diff = read_and_synchronize_bag_files(bag_file_master, bag_file_slave)
wait_key = 1
post_process = False

try:
    while True:
        # Get frameset of color and depth
        frames_master = pipe_master.wait_for_frames()
        frames_slave = pipe_slave.wait_for_frames()

        # Process the frames
        depth_frame_master = frames_master.get_depth_frame()
        depth_frame_slave = frames_slave.get_depth_frame()

        # Validate that both frames are valid
        if not depth_frame_master or not depth_frame_slave:
            continue

        # Post process is not included in the BAG file, so we need to apply it
        if post_process:
            depth_frame_master = post_process_depth_frame(depth_frame_master)
            depth_frame_slave = post_process_depth_frame(depth_frame_slave)

        # Colorize the depth frame
        depth_color_frame_master = colorizer.colorize(depth_frame_master)
        depth_color_frame_slave = colorizer.colorize(depth_frame_slave)

        # Convert frames to images
        depth_color_image_master = np.asanyarray(depth_color_frame_master.get_data())
        depth_color_image_slave = np.asanyarray(depth_color_frame_slave.get_data())
        
        # Stack the images horizontally
        frame_shape = (depth_frame_master.get_height(), depth_frame_master.get_width())
        new_frame_shape = (int(frame_shape[1] / 1.5), int(frame_shape[0] / 1.5))
        depth_color_image_master = cv2.resize(depth_color_image_master, new_frame_shape)
        depth_color_image_slave = cv2.resize(depth_color_image_slave, new_frame_shape)
        images = np.hstack((depth_color_image_master, depth_color_image_slave))
        cv2.imshow("Depth Images", images)

        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press d to view the video frame by frame
        if key == ord('d'):
            wait_key = 1 if wait_key == 0 else 0

# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")

# Release the stream 
finally:
    cv2.destroyAllWindows()
    pipe_master.stop()
    pipe_slave.stop()

## 2. Ground projection comparison

Compute Homography for the first camera

In [26]:
points_floor_cam1 = np.array([
    [267, 140], [369, 133], [473, 132], 
    [267, 218], [374, 210], [472, 206], 
    [269, 301], [377, 298], [479, 302]
])

points_100_cam1 = np.array([
    [167, 71], [333, 60], [510, 57], 
    [169, 200], [346, 188], [510, 182],
    [168, 340], [351, 334], [529, 340]
])

In [27]:
points_floor_file1 = "C:\\Users\\Robin\\Documents\\Stage2024\\Dataset\\points_measurements_projection\\artficial_experiment\\points_ground_1\\0.png"
image = cv2.imread(points_floor_file1)

for pf, p100 in zip(points_floor_cam1, points_100_cam1):
    cv2.line(image, pf[:2], p100[:2], (255, 0, 0), 1)

cv2.namedWindow('Projection Lines', cv2.WINDOW_NORMAL)
cv2.imshow('Projection Lines', image)
cv2.waitKey()
cv2.destroyAllWindows()

In [28]:
# Compute the 2 homographies
H_0_cam1, _ = cv2.findHomography(points_floor_cam1, points_floor_cam1) # Identity matrix (hopefully :))
H_100_cam1, _ = cv2.findHomography(points_100_cam1, points_floor_cam1)

In [29]:
def interpolate_homography2(z, H_0, H_100):
    """
    Interpolate the homography matrix for a given height, between 0 and 200 cm.
    """
    if z <= 100: # linear interpolation between 0 and 100
        alpha = z / 100
        return (1 - alpha) * H_0 + alpha * H_100
    
    else: # extrapolation between 100 and 200
        H_200 =  H_100 + (H_100 - H_0)
        alpha = (z - 100) / 100
        return (1 - alpha) * H_100 + alpha * H_200

In [30]:
# Small example to test the homography interpolation
image = cv2.imread(points_floor_file1)

# Draw the points at 100 cm
for point in points_100_cam1.astype(int):
    cv2.circle(image, tuple(point), 3, (255, 0, 0), -1)

cv2.namedWindow('Projection Points', cv2.WINDOW_NORMAL)
cv2.imshow('Projection Points', image)

def on_trackbar(z):
    z = int(z)
    H = interpolate_homography2(z, H_0_cam1, H_100_cam1)

    projected_points = []

    for point in points_100_cam1:
        projected_point = project_point(point, z, H)
        projected_points.append(projected_point)

    image_copy = image.copy()
    for point in projected_points:
        point = point.astype(int)
        cv2.circle(image_copy, tuple(point), 3, (0, 0, 255), -1)

    cv2.imshow('Projection Points', image_copy)

cv2.createTrackbar('Height', 'Projection Points', 100, 200, on_trackbar)

cv2.waitKey(0)
cv2.destroyAllWindows()

Compute Homography for the second camera

In [31]:
points_floor_cam2 = np.array([
    [473, 164], [471, 240], [473, 333], 
    [373, 329], [370, 241], [365, 159], 
    [263, 165], [263, 245], [263, 329]
])

points_100_cam2 = np.array([
    [513, 121], [508, 249], [519, 406], 
    [345, 395], [342, 251], [331, 120], 
    [159, 125], [161, 259], [162, 399]
])

In [32]:
points_floor_file2 = "C:\\Users\\Robin\\Documents\\Stage2024\\Dataset\\points_measurements_projection\\artficial_experiment\\points_ground_2\\0.png"
image = cv2.imread(points_floor_file2)

for pf, p100 in zip(points_floor_cam2, points_100_cam2):
    cv2.line(image, pf[:2], p100[:2], (255, 0, 0), 1)

cv2.namedWindow('Projection Lines', cv2.WINDOW_NORMAL)
cv2.imshow('Projection Lines', image)
cv2.waitKey()
cv2.destroyAllWindows()

In [33]:
# Compute the 2 homographies
H_0_cam2, _ = cv2.findHomography(points_floor_cam2, points_floor_cam2) # Identity matrix (hopefully :))
H_100_cam2, _ = cv2.findHomography(points_100_cam2, points_floor_cam2)

In [41]:
# Small example to test the homography interpolation
image = cv2.imread(points_floor_file2)

# Draw the points at 100 cm
for point in points_100_cam2.astype(int):
    cv2.circle(image, tuple(point), 3, (255, 0, 0), -1)

cv2.namedWindow('Projection Points', cv2.WINDOW_NORMAL)
cv2.imshow('Projection Points', image)

def on_trackbar(z):
    z = int(z)
    H = interpolate_homography2(z, H_0_cam2, H_100_cam2)

    projected_points = []

    for point in points_100_cam2:
        projected_point = project_point(point, z, H)
        projected_points.append(projected_point)

    image_copy = image.copy()
    for point in projected_points:
        point = point.astype(int)
        cv2.circle(image_copy, tuple(point), 3, (0, 0, 255), -1)

    cv2.imshow('Projection Points', image_copy)

cv2.createTrackbar('Height', 'Projection Points', 100, 200, on_trackbar)

cv2.waitKey(0)
cv2.destroyAllWindows()

Compare ground projections

In [42]:
bag_file_master = "C:\\Users\\Robin\\Documents\\Stage2024\\Dataset\\bag_records\\artificial_exit_line\\robin_master.bag"

bag_file_slave = "C:\\Users\\Robin\\Documents\\Stage2024\\Dataset\\bag_records\\artificial_exit_line\\robin_slave.bag"

pipe_master, pipe_slave, timestamp_diff = read_and_synchronize_bag_files(bag_file_master, bag_file_slave)
print("Timestamp difference: {} ms".format(timestamp_diff))

Timestamp difference: 28.009033203125 ms


In [43]:
def process_frames(frames_master, frames_slave):
    # Process the frames
    infrared_frame_master = frames_master.get_infrared_frame()
    infrared_image_master = np.asanyarray(infrared_frame_master.get_data())
    image_master = cv2.cvtColor(infrared_image_master, cv2.COLOR_GRAY2BGR)

    infrared_frame_slave = frames_slave.get_infrared_frame()
    infrared_image_slave = np.asanyarray(infrared_frame_slave.get_data())
    image_slave = cv2.cvtColor(infrared_image_slave, cv2.COLOR_GRAY2BGR)

    return image_master, image_slave

def detect_markers_and_project(image, detector, H0, H100, height, projected_points):
    # Detect the aruco markers
    corners, ids, rejected = detector.detectMarkers(image)
    cv2.aruco.drawDetectedMarkers(image, corners, ids)

    for k in range(len(corners)):
        id = ids[k][0]
        c = corners[k][0]

        # Center of the aruco marker
        x = int(c[:, 0].sum() / 4)
        y = int(c[:, 1].sum() / 4)

        # Project the point on the floor
        point = (x, y)
        H = interpolate_homography2(height, H0, H100)
        projected_point = project_point(point, height, H)
        projected_points.append(projected_point)

        cv2.circle(image, point, 4, (0, 0, 255), -1)

    return image, projected_points

In [44]:
wait_key = 1

height = 180
projected_points_cam1 = []
projected_points_cam2 = []

def clear_points(val):
    if val == 1:
        projected_points_cam1.clear()
        projected_points_cam2.clear()

def get_pos(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print("(x : {}, y : {})".format(x, y))

cv2.namedWindow('Frame')
cv2.setMouseCallback('Frame', get_pos)
cv2.createTrackbar('Clear', 'Frame', 0, 1, clear_points)

try:
    while True:
        frames_master = pipe_master.wait_for_frames()
        frames_slave = pipe_slave.wait_for_frames()

        # Process the frames
        image_master, image_slave = process_frames(frames_master, frames_slave)

        # Detect the aruco markers and project the points
        image_master, projected_points_cam1 = detect_markers_and_project(image_master, detector, H_0_cam2, H_100_cam2, height, projected_points_cam1)
        image_slave, projected_points_cam2 = detect_markers_and_project(image_slave, detector, H_0_cam1, H_100_cam1, height, projected_points_cam2)
        
        # Draw all the projected points to get the trajectory
        for projected_point in projected_points_cam1:
            point = projected_point.astype(int)
            cv2.circle(image_master, tuple(point), 4, (0, 255, 255), -1)

        for projected_point in projected_points_cam2:
            point = projected_point.astype(int)
            cv2.circle(image_slave, tuple(point), 4, (0, 255, 255), -1)

        # resize images with ratio 0.75
        image_master = cv2.resize(image_master, (0, 0), fx=0.75, fy=0.75)
        y_exit_master = 213
        cv2.line(image_master, (0, y_exit_master), (image_master.shape[1], y_exit_master), (0, 0, 255), 1)

        image_slave = cv2.resize(image_slave, (0, 0), fx=0.75, fy=0.75)
        y_exit_slave = 189
        cv2.line(image_slave, (0, y_exit_slave), (image_slave.shape[1], y_exit_slave), (0, 0, 255), 1)

        images = np.hstack((image_master, image_slave))
        cv2.imshow('Frame', images)
        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press d to view the video frame by frame
        if key == ord('d'):
            wait_key = 1 if wait_key == 0 else 0
           
# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")

# Release the stream    
finally:
    cv2.destroyAllWindows()
    pipe_master.stop()
    pipe_slave.stop()